#Modelo con TensorFlow OD API

In [ ]:
#Importamos librerias Necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import os
import re
import cv2
import random
from PIL import Image, ImageFilter, ImageDraw, ImageFont
#from IPython.display import Image as ImageDisplay
#from IPython.core.display import HTML
#from IPython.display import display, Image as IPImage
#from io import BytesIO
#import base64
#from sklearn.model_selection import train_test_split
import tensorflow as tf

2025-06-22 13:14:12.787001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750608852.965644  363764 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750608853.016311  363764 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750608853.453462  363764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750608853.453506  363764 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750608853.453508  363764 computation_placer.cc:177] computation placer alr

In [ ]:
imagenes_np = np.load('/home/garzamorada/my_character_detector_workspace/ocr_font_images_dataset.npy', allow_pickle=True)
imagenes_df = pd.read_csv('/home/garzamorada/my_character_detector_workspace/ocr_font_labels_dataset.csv')
path_tfrecord = '/home/garzamorada/my_character_detector_workspace/eval/tfrecord'
os.makedirs(path_tfrecord, exist_ok=True)

In [ ]:
caracteres = "ABCDEFGHIJKLMNOPQRSTUVWXYZÑabcdefghijklmnopqrstuvwxyzñáéíóúü0123456789.,;:?¿!¡-—&*/+=-_@#$%"

# Eliminar duplicados preservando orden
caracteres = ''.join(dict.fromkeys(caracteres))

label_list = [{"id": idx, "label": c} for idx, c in enumerate(caracteres, start=1)]


def get_id_by_char(label, labels=label_list):
    for item in labels:
        if item["label"] == label:
            return item["id"]
    return None

def generar_label_map(label_list, ruta_archivo):
    with open(ruta_archivo, 'w', encoding='utf-8') as f:
        for item in label_list:
            f.write("item {\n")
            f.write(f"  id: {item['id']}\n")
            f.write(f"  name: '{item['label']}'\n")
            f.write("}\n")

generar_label_map(label_list, "/home/garzamorada/my_character_detector_workspace/label_map.pbtxt")

## Generador de imagenes

In [ ]:
# Generador de tfrecord
def guardar_imagen_y_tfrecord(imagen_np, bboxes, labels, chars, output_dir, nombre_base='img', idx=0):
    """
    Guarda una imagen en disco y genera un archivo TFRecord correspondiente.

    Args:
        imagen_np: np.ndarray (H, W) en escala de grises.
        bboxes: lista de bboxes en formato [xmin, ymin, xmax, ymax].
        labels: lista de strings de etiquetas correspondientes a cada bbox.
        output_dir: carpeta donde guardar los archivos.
        nombre_base: nombre base para los archivos (ej: 'img' → img_001.png).
        idx: índice numérico único para el nombre de archivo.
    """
    os.makedirs(output_dir, exist_ok=True)

    # 1. Guardar imagen en PNG
    filename = f'{nombre_base}_{idx:04d}.png'
    path_imagen = os.path.join(output_dir, filename)
    Image.fromarray(imagen_np).save(path_imagen)

    # 2. Codificar imagen
    with tf.io.gfile.GFile(path_imagen, 'rb') as fid:
        encoded_image_data = fid.read()
    height, width = imagen_np.shape

    # 3. Normalizar bboxes
    xmins = [box[0] / width for box in bboxes]
    xmaxs = [box[2] / width for box in bboxes]
    ymins = [box[1] / height for box in bboxes]
    ymaxs = [box[3] / height for box in bboxes]
    classes_text = [char.encode('utf8') for char in chars]
    classes_ids = labels

    # 4. Crear TF Example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'png'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes_ids)),
    }))

    # 5. Guardar TFRecord
    tfrecord_path = os.path.join(output_dir, f'{nombre_base}_{idx:04d}.tfrecord')
    with tf.io.TFRecordWriter(tfrecord_path) as writer:
        writer.write(tf_example.SerializeToString())

    return path_imagen, tfrecord_path

In [ ]:
def redimensionar_imagen_y_bbox(imagen_cv2, bbox, nuevo_ancho, nuevo_alto):

    # Obtener las dimensiones originales de la imagen
    alto_original, ancho_original = imagen_cv2.shape[:2]

    # Redimensionar la imagen
    imagen_redimensionada = cv2.resize(imagen_cv2, (nuevo_ancho, nuevo_alto), interpolation=cv2.INTER_AREA)

    # Calcular los factores de escala
    factor_escala_ancho = nuevo_ancho / ancho_original
    factor_escala_alto = nuevo_alto / alto_original

    # Ajustar las coordenadas del bounding box
    # Asumimos que bbox viene como (x_min, y_min, x_max, y_max)
    x_min_original, y_min_original, x_max_original, y_max_original = bbox

    x_min_ajustado = int(x_min_original * factor_escala_ancho)
    y_min_ajustado = int(y_min_original * factor_escala_alto)
    x_max_ajustado = int(x_max_original * factor_escala_ancho)
    y_max_ajustado = int(y_max_original * factor_escala_alto)

    bbox_ajustado = (x_min_ajustado, y_min_ajustado, x_max_ajustado, y_max_ajustado)

    return imagen_redimensionada, bbox_ajustado

In [ ]:
def generate_dataset(
    original_numpy_dataset,
    original_pandas_dataset,
    large_img_size=(320, 320),
    char_size=(128, 128), # Este es el tamaño fijo de tus caracteres originales
):
    global_composed_index = 0
    output_img_count = len(original_pandas_dataset)

    new_w, new_h = large_img_size

    if original_numpy_dataset.size == 0:
        print("El dataset de caracteres original está vacío. No se pueden generar imágenes compuestas.")
        return np.array([], dtype=[('image', object), ('bboxes', object), ('index', int)]), pd.DataFrame()

    for i in range(output_img_count):
        if ((global_composed_index + 1) * 100) % output_img_count == 0:
            print(f"realizado {((global_composed_index + 1) * 100) // output_img_count}% ")

        #char_entry = original_numpy_dataset[i]
        #char_index = char_entry['index']
        #char_data = original_pandas_dataset[original_pandas_dataset['index'] == char_index].iloc[0]
        char_data = original_pandas_dataset.iloc[i]
        index = char_data['index']
        char_entry = original_numpy_dataset[original_numpy_dataset['index'] == index][0]

        char_label = char_data['character']
        id=get_id_by_char(char_label)
        char_image = char_entry['image']
        original_char_bbox = char_entry['bbox']
        new_char_image, new_char_bbox = redimensionar_imagen_y_bbox(char_image, original_char_bbox, new_w, new_h)

        current_image_bboxes = []
        labels=[]
        chars=[]
        chars.append(char_label)
        labels.append(id)
        current_image_bboxes.append(new_char_bbox)
        ruta_imagen, ruta_record = guardar_imagen_y_tfrecord(new_char_image, current_image_bboxes, labels, chars, path_tfrecord, nombre_base='img', idx=global_composed_index)

        global_composed_index += 1

In [ ]:
# Parámetros para la generación de imágenes compuestas
#num_composed_images_to_generate = 5000 # Define cuántas imágenes grandes quieres generar
output_large_img_size = (320, 320)
input_char_size = (128, 128) # Tamaño fijo de tus caracteres de origen
original_numpy_dataset= imagenes_np
#original_pandas_dataset= imagenes_df.sample(5000) #Para eval
original_pandas_dataset= imagenes_df

generate_dataset(
    original_numpy_dataset,
    original_pandas_dataset,
    large_img_size=output_large_img_size,
    char_size=input_char_size
)

realizado 1% 
realizado 2% 
realizado 3% 
realizado 4% 
realizado 5% 
realizado 6% 
realizado 7% 
realizado 8% 
realizado 9% 
realizado 10% 
realizado 11% 
realizado 12% 
realizado 13% 
realizado 14% 
realizado 15% 
realizado 16% 
realizado 17% 
realizado 18% 
realizado 19% 
realizado 20% 
realizado 21% 
realizado 22% 
realizado 23% 
realizado 24% 
realizado 25% 
realizado 26% 
realizado 27% 
realizado 28% 
realizado 29% 
realizado 30% 
realizado 31% 
realizado 32% 
realizado 33% 
realizado 34% 
realizado 35% 
realizado 36% 
realizado 37% 
realizado 38% 
realizado 39% 
realizado 40% 
realizado 41% 
realizado 42% 
realizado 43% 
realizado 44% 
realizado 45% 
realizado 46% 
realizado 47% 
realizado 48% 
realizado 49% 
realizado 50% 
realizado 51% 
realizado 52% 
realizado 53% 
realizado 54% 
realizado 55% 
realizado 56% 
realizado 57% 
realizado 58% 
realizado 59% 
realizado 60% 
realizado 61% 
realizado 62% 
realizado 63% 
realizado 64% 
realizado 65% 
realizado 66% 
realizado 67% 
real

In [ ]:
print(format(label_list))
print(f"largo: {len(label_list)}")